In [1]:
# import dependencies
import pandas as pd
import mysql.connector as mc
from datetime import datetime
import numpy as np
from bs4 import BeautifulSoup as Soup
import pdfkit
import win32com.client as wc
import xml.etree.ElementTree as ET
import sys
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import ssl
import smtplib
import os

# Get db credentials
from configTest import mysql_host, mysql_u, mysql_pw, vgc_host, vgc_u, vgc_pw, smtp_host, e_user, e_pw, port

In [2]:
# MySQL
def mysql_q (u, p, h, db, sql, cols, commit):
    # cols (0 = no, 1 = yes)
    # commit (select = 0, insert/update = 1)

    # connect to claim_qb_payments db
    cnx = mc.connect(user=u, password=p,
                    host=h,
                    database=db)
    cursor = cnx.cursor()

    # commit?
    if commit == 1:
        cursor.execute(sql)
        cnx.commit()
        sql_result = 0     
    else:
        cursor.execute(sql)
        sql_result = cursor.fetchall()

    # columns ?
    if cols == 1:
        columns=list([x[0] for x in cursor.description])
        # close connection
        cursor.close()
        cnx.close()
        # return query result [0] and columns [1]
        return sql_result
    else:
        # close connection
        cursor.close()
        cnx.close()
        # return query result
        return sql_result 

In [3]:
# Email
def send_email(toEmail, subject, msg_html, attachPath='', *args):
    fromEmail = 'claims@visualgap.com'

    # address message
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = fromEmail
    msg['To'] = ','.join(toEmail)

    # create body
    body_html = MIMEText(msg_html, 'html')
    msg.attach(body_html) 

    for arg in args:
        fName = ''.join([attachPath, arg])
        filename = os.path.abspath(fName)

        with open(filename, 'rb') as fn:
            attachment = MIMEApplication(fn.read())
            attachment.add_header('Content-Disposition', 'attachment', filename=arg)
            msg.attach(attachment)

    context = ssl.create_default_context()
    try:
        server = smtplib.SMTP(smtp_host, port)
        # check connection
        server.ehlo()  
        # Secure the connection
        server.starttls(context=context)  
        # check connection
        server.ehlo()
        server.login(e_user, e_pw)
        # Send email
        server.sendmail(fromEmail, toEmail, msg.as_string())

    except Exception as e:
        # Print any error messages
        print(e)
    finally:
        server.quit()

In [4]:
# Define directories
attachment_dir = 'S:/claims/letters/attachment/'
file_staging_dir = './letters/staging/'

now = datetime.now()

In [5]:
# Define file paths
attach_name = f'Claims_Paid_{now.strftime("%Y-%m-%d")}'
attach_file = f'{attachment_dir}{attach_name}'
html_file = f'{file_staging_dir}final_rpt.html'
err_html_file = f'{file_staging_dir}error.html'
csv_file = f'{file_staging_dir}final_rpt.csv'
err_csv_file = f'{file_staging_dir}error.csv'
template_file = './html/pymt_summary_template.html'
html_file2 = './letters/staging/Claims_Paid.html'

In [6]:
# sql query to collect qb_txnid's
sql = '''
      SELECT rtbp_id, check_nbr, qb_txnid
      FROM ready_to_be_paid
      WHERE toVGC = 1
        AND qb_txnid <> '0';
      '''

In [7]:
# save query results as DF
qb_df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql, 0, 0))

# add column names
qb_df_cols = ['rtbp_id','check_nbr', 'qb_txnid']
qb_df.columns = qb_df_cols

In [8]:
# Connect to Quickbooks
try:
    sessionManager = wc.Dispatch("QBXMLRP2.RequestProcessor")    
    sessionManager.OpenConnection('', 'Claim Payments')
    ticket = sessionManager.BeginSession("", 2)
except Exception as e:
    print('''
    Make sure QuickBooks is running and you are logged into the Company File.
    ERROR: {}'''.format(e))
    sys.exit("Error with communicating with QuickBooks")

In [9]:
# create qbxml to query qb for check numbers
try:
    for index, row in qb_df.iterrows():
        qbxmlQuery = '''
                    <?qbxml version="14.0"?>
                    <QBXML>
                        <QBXMLMsgsRq onError="stopOnError">
                            <CheckQueryRq>
                                <TxnID>{txnId}</TxnID> 
                            </CheckQueryRq>
                        </QBXMLMsgsRq>
                    </QBXML>
                    '''.format(txnId=row['qb_txnid'])

        # Send query and receive response
        responseString = sessionManager.ProcessRequest(ticket, qbxmlQuery)

        # output Check Number (RefNumber)
        QBXML = ET.fromstring(responseString)
        QBXMLMsgsRs = QBXML.find('QBXMLMsgsRs')
        checkResults = QBXMLMsgsRs.iter("CheckRet")
        chkNbr = '0'
        for checkResult in checkResults:
            chkNbr = checkResult.find('RefNumber').text

        # Add Check Number to ready_to_be_paid table
        qb_sql_file = '''UPDATE ready_to_be_paid
                    SET check_nbr = '{ChkNbr}'
                    WHERE rtbp_id = {rowID};'''.format(ChkNbr=chkNbr, rowID=row['rtbp_id'])
        
        # execute and commit sql
        mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', qb_sql_file, 0, 1)
except Exception as e:
    print('''
    Make sure to print checks and process ACH in Quickbooks prior to starting this process.
    ERROR: {}'''.format(e))
    sys.exit("Error with communicating with QuickBooks")

In [10]:
# Disconnect from Quickbooks
sessionManager.EndSession(ticket)
sessionManager.CloseConnection()

In [11]:
# sql query for GAP claims that are RTBP
sql = '''
      SELECT r.rtbp_id, r.claim_id, r.claim_nbr, r.carrier_id, r.lender_name, r.pymt_method, r.first, r.last, r.pymt_type_id, r.amount, r.payment_category_id, r.check_nbr, r.qb_txnid, r.pymt_date, r.toVGC, r.err_msg
      FROM ready_to_be_paid r
      INNER JOIN (SELECT batch_id
                  FROM ready_to_be_paid
                  WHERE toVGC = 1
                  GROUP BY batch_id) sq
      USING(batch_id);
      '''

In [12]:
# save query results as DF
df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql, 0, 0))

In [13]:
# add column names
df_cols = ['rtbp_id', 'claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'payment_category_id', 'check_nbr', 'qb_txnid', 'pymt_date', 'toVGC', 'err_msg']

df.columns = df_cols

In [14]:
# Add carrier name
sql = '''
    SELECT carrier_id, description
    FROM carriers;
    '''
# save query results as DF
carrier_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql, 0, 0))

col_names = ['carrier_id', 'carrier']
carrier_df.columns = col_names

# Merge QB_ListID into df
df = df.merge(carrier_df, left_on='carrier_id', right_on='carrier_id').copy()


In [15]:
# format df
# list of conditions
type_conds = [((df['payment_category_id'] == 1) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 1) & (df['pymt_type_id'] == 2)),
              ((df['payment_category_id'] == 2) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 2) & (df['pymt_type_id'] == 2)),
              ((df['payment_category_id'] == 3) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 3) & (df['pymt_type_id'] == 2)),]
# list of name types
type_name = ['GAP', 'GAP Supp', 'GAP Plus', 'GAP Plus Supp', 'TotalRestart', 'TotalRestart Supp']

# add column and assigned values
df['Claim_Type'] = np.select(type_conds, type_name)


In [16]:
# Create df to update VGC
# vgc_update_df = df[['claim_id', 'pymt_method', 'pymt_type_id', 'amount', 'payment_category_id', 'check_nbr', 'qb_txnid', 'pymt_date']].copy()

In [17]:
# Create df for Claim Payment Summary
final_rpt_df = df[['claim_nbr', 'carrier', 'lender_name', 'first', 'last', 'amount', 'pymt_method', 'check_nbr', 'pymt_date', 'Claim_Type']].loc[df['toVGC'] == 1].copy()

In [18]:
# Format df
final_rpt_df.rename(columns = {'claim_nbr':'Claim Nbr', 'carrier':'Carrier', 'lender_name':'Lender', 'first':'First Name', 'last':'Last Name', 'amount':'Amount', 'pymt_method':'Method',
                               'check_nbr':'Check Nbr', 'pymt_date': 'Date', 'Claim_Type':'Claim Type'}, inplace=True)
                               
final_rpt_df['Amount'] = final_rpt_df['Amount'].map('${:,.2f}'.format)

In [19]:
final_rpt_df.sort_values(by = ['Carrier', 'Last Name', 'First Name'], inplace=True)

In [20]:
# create error_df
error_df = df[['claim_nbr', 'lender_name', 'first', 'last', 'err_msg']].loc[df['toVGC'] == 2].copy()

# Format df
error_df.rename(columns = {'claim_nbr':'Claim Nbr', 'lender_name':'Lender', 'first':'First Name', 'last':'Last Name', 'err_msg':'Error Message'}, inplace=True)

In [21]:
error_df

,Claim Nbr,Lender,First Name,Last Name,Error Message
20,202112272514,Charlotte Metro Credit Union,Supplemental,Letter,No matching CHECKING and-or EXPENSE accounts i...


In [22]:
# export is df as csv
final_rpt_df.to_csv(csv_file, index=False)
error_df.to_csv(err_csv_file, index=False)
# read in csv
csvFile = pd.read_csv(csv_file)
errCsvFile = pd.read_csv(err_csv_file)
# convert csv to html
csvFile.to_html(html_file, index=False)
errCsvFile.to_html(err_html_file, index=False)

In [23]:
# Get html df
soup = Soup(open(html_file), "html.parser")
err_soup = Soup(open(err_html_file), "html.parser")
table = str(soup.select_one("table", {"class":"dataframe"}))
err_table = str(err_soup.select_one("table", {"class":"dataframe"}))

# Get template
soup2 = Soup(open(template_file), "html.parser")
# Find and insert payment table
df_div = soup2.find("div", {"id":"df"})
df_div.append(Soup(table, 'html.parser'))
# Find and insert error table
err_div = soup2.find("div", {"id":"error"})
err_div.append(Soup(err_table, 'html.parser'))

# write html file
with open(html_file2,'w') as file:
    file.write(str(soup2))

In [24]:
# check if file exists
fName = ''.join([attach_file, '.pdf'])
fnNum = 0

while(os.path.isfile(fName) == True):
    fnNum += 1
    fName = ''.join([attach_file, '_', str(fnNum), '.pdf']) 


In [25]:
# create PDF from html
pdf_options = {'orientation': 'landscape',
                'page-size': 'Letter',
                'margin-top': '0.25in',
                'margin-right': '0.25in',
                'margin-bottom': '0.25in',
                'margin-left': '0.25in',
                'encoding': "UTF-8",}
                
pdfkit.from_file(html_file2, fName, options=pdf_options)

True

In [26]:
# Email Claim Summary Report
to = ['jared@visualgap.com']
sub = f'Claim Payment Summary {now.strftime("%Y-%m-%d")}'
msg_html = '''
            <html>
            <body>
                <p>Attached is the Claim Summary Report.<br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
           '''
if fnNum == 0:
    a_file = ''.join([attach_name, '.pdf'])
else:
    a_file = ''.join([attach_name, '_', str(fnNum), '.pdf'])

send_email(to, sub, msg_html, attachment_dir, a_file)

In [27]:
# Create data file for SCC
 

In [28]:
# Send file to SCC via SFTP

In [29]:
# Send Email notification to SCC and Claims
to = ['jared@visualgap.com']
sub = f'Frost claim file'
msg_html = '''
            <html>
            <body>
                <p>Hello,<br>
                <br>
                We have submitted a new claim file today.  If you have any questions or concerns please contact us. <br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
           '''
send_email(to, sub, msg_html)

In [31]:
# update toVGC to 3
# if len(df) > 0:
#     for index, row in df.iterrows():
#         err_sql = '''
#                 UPDATE ready_to_be_paid
#                 SET toVGC = 3
#                 WHERE rtbp_id = {rtbp_id};
#                 '''.format(rtbp_id=row['rtbp_id'])
#         # run update query        
#         mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', err_sql, 0, 1)